<a href="https://colab.research.google.com/github/ashokreddy123/session3_pytoroch/blob/main/session3_assignment_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing the required libraries
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [2]:
# downloading the FashionMnist dataset

train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)




Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [185]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

In [4]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [5]:
# dividing the data into training and validation data
# out of 60,000 images, 59,000 are considered for training and 1000 for validation set
count  = 0
training_data = []
validation_data = []
for batch in train_loader:
  count += 1
  if(count<591):
    training_data.append(batch)
  else:
    validation_data.append(batch)

In [195]:
print("training set size",len(training_data)*batch_size)
print("validation set size",len(validation_data)*batch_size)

training set size 59000
validation set size 1000


In [186]:
# All the convolutional layers are of 3x3 kernel size
# All the maxpool layers are of 2x2 size with stride 2

import torch.nn.functional as F
import torch.nn as nn
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        #self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=64 * 4 * 4, out_features=64*4)
        self.fc2 = nn.Linear(in_features=64*4, out_features=64)
        self.out1 = nn.Linear(in_features=64, out_features=10) # This is for the number output from MNIST data
        self.out2 = nn.Linear(in_features=20, out_features=64) # This layer stacks the number output from MNIST data set and one hot code of random number
        self.out3 = nn.Linear(in_features=64, out_features=20) # The sum output(of size 20)

    def forward(self, t, rand_num):
       
        t = self.conv1(t)
        t = F.relu(t)
        t = self.conv2(t)
        t = F.relu(t)
        #t = t.cuda()
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        #t = t.cuda()

        # (3) hidden conv layer
        t = self.conv3(t)
        t = F.relu(t)
        t = self.conv4(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 64 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t1 = self.out1(t)
        #print(t1)
        t2 = t1.argmax(dim=1) # The position where MNIST ouput is max is taken
        #print("1_t2.shape",t2.shape)
        t2 = F.one_hot(t2, num_classes=10) # The MNIST output is converted to one hot encoding
        #print("2_t2.shape",t2.shape)
        t_rand = F.one_hot(torch.tensor(rand_num),num_classes=10) # the random output is converted to one hot encoding
        #print("3_t_rand.shape",t_rand.shape)
        #t_rand = t_rand.unsqueeze(0)
        t_both = torch.stack((t2,t_rand),dim=1) # The MNIST output encoding and random output encoding are stacked 
        #print("4_t_both.shape",t_both.shape)
        t_both = t_both.reshape(batch_size,-1)
        #print("5_t_both.shape",t_both.shape)
        t_both = torch.tensor(t_both,dtype=torch.float32)
        #print("6_t_both.shape",t_both.shape)
        #print(t_both.shape)
        t_both = self.out2(t_both) # The stacked output is connected to FC layers
        t_both = F.relu(t_both)

        t_both = self.out3(t_both) # The final sum output of size 20
        #t_both = F.relu(t_both)

        #t = F.softmax(t)
        #t = F.softmax(t, dim=1)
        #t = t.argmax(dim=1)

        return t1,t_both

In [188]:
import torch.optim as optim
network = Network()
network = network.cuda() # The network is moved to GPU
torch.set_grad_enabled(True)
optimizer = optim.Adam(network.parameters(), lr=0.01) # Adam optimier is used
#print(optimizer.device)

In [126]:
# This is to give MNIST detection and sum accuracy
def get_num_correct2(pred1, pred2, labels,rand_num):
  return pred1.argmax(dim=1).eq(labels).sum().item(),pred2.argmax(dim=1).eq(labels+rand_num).sum().item() 

In [191]:
import random
for i in range(50):
  total_correct_detect = 0
  total_correct_sum = 0
  #print("total_loss",total_loss.device)
  for batch in training_data:
    images,labels = batch
    #print("cpu_device",images.device)
    images,labels = images.cuda(),labels.cuda() # The data is moved to GPU
    #print(labels.shape)
    rand_num = []
    for j in range(batch_size):
      rand_num.append(random.randint(0,9)) # creating the rand numbers of batch size
    #print(len(rand_num))
    rand_num = torch.tensor(rand_num)
    rand_num = rand_num.cuda()
    
    #print(rand_num.shape)
    #rand_num = rand_num.reshape(100,1)
    #print("Gpu_device",images.device)
    #print(labels)
    #preds = network(images)
    pred1,pred2 =  network(images,rand_num) # pred1 is predictions of MNIST data and pred2 is for sum 
    #print(preds.shape)
    #print(labels.shape)
    #print(preds.device)
    #print("accuracy :",get_num_correct(preds, labels))

    #loss = F.cross_entropy(preds,labels)
    #print("labels",labels)
    #print("rand_num",rand_num)
    #print('sum',labels+rand_num)
    loss1 = F.cross_entropy(pred1,labels) # loss for MNIST detection
    loss2 = F.cross_entropy(pred2,labels+rand_num) # loss for sum accuracy
    #print("loss1",loss1)
    #print("loss2",loss2)
    loss = loss1 + loss2 # total loss is the sum of loss
    correct_detect, correct_sum = get_num_correct2(pred1, pred2, labels,rand_num)
    total_correct_detect += correct_detect
    total_correct_sum += correct_sum
    total_loss += loss
    #print("total_loss",total_loss.device)
    #print("loss",loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("epoch",i+1,"loss",total_loss,"detection_accuracy :",total_correct_detect*100/(len(training_data)*batch_size),"sum_accuracy :",total_correct_sum*100/(len(training_data)*batch_size))
  #print("epoch",i+1,"loss",total_loss,"accuracy :",total_correct_sum*100/(len(training_data)*batch_size))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch 1 loss tensor(174584.8594, device='cuda:0', grad_fn=<AddBackward0>) detection_accuracy : 80.72033898305085 sum_accuracy : 80.6457627118644
epoch 2 loss tensor(175137.7500, device='cuda:0', grad_fn=<AddBackward0>) detection_accuracy : 85.61016949152543 sum_accuracy : 85.61016949152543
epoch 3 loss tensor(175652.7656, device='cuda:0', grad_fn=<AddBackward0>) detection_accuracy : 86.6728813559322 sum_accuracy : 86.6728813559322
epoch 4 loss tensor(176146.2969, device='cuda:0', grad_fn=<AddBackward0>) detection_accuracy : 87.27966101694915 sum_accuracy : 87.27966101694915
epoch 5 loss tensor(176635.1250, device='cuda:0', grad_fn=<AddBackward0>) detection_accuracy : 87.4593220338983 sum_accuracy : 87.45423728813559
epoch 6 loss tensor(177110.1094, device='cuda:0', grad_fn=<AddBackward0>) detection_accuracy : 87.85593220338983 sum_accuracy : 87.85593220338983
epoch 7 loss tensor(177585.2812, device='cuda:0', grad_fn=<AddBackward0>) detection_accuracy : 87.81694915254238 sum_accuracy : 

In [192]:
# the accuracy for validation data
total_val_detect_correct = 0
total_val_sum_correct = 0
#total_loss = 0
for batch in validation_data:
  images,labels = batch
  images,labels = images.cuda(),labels.cuda()
  #print(labels)
  #preds = network(images)
  rand_num = []
  for j in range(batch_size):
    rand_num.append(random.randint(0,9))
    #print(len(rand_num))
  rand_num = torch.tensor(rand_num)
  rand_num = rand_num.cuda()
  pred1,pred2 =  network(images,rand_num)
  correct_detect, correct_sum = get_num_correct2(pred1, pred2, labels,rand_num)

  total_val_detect_correct += correct_detect
    
  total_val_sum_correct += correct_sum
  

print("val_detect_accuracy :",total_val_detect_correct*100/(len(validation_data)*batch_size),"val_sum_accuracy",total_val_sum_correct*100/(len(validation_data)*batch_size))

val_detect_accuracy : 87.3 val_sum_accuracy 87.3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
